In [8]:
'''

This code is part of the SIPN2 project focused on improving sub-seasonal to seasonal predictions of Arctic Sea Ice. 
If you use this code for a publication or presentation, please cite the reference in the README.md on the
main page (https://github.com/NicWayand/ESIO). 

Questions or comments should be addressed to nicway@uw.edu

Copyright (c) 2018 Nic Wayand

GNU General Public License v3.0


'''

'''
weekly average, compute metrics, save for all available models and obs
redo for last two weeks each time called, takes a long time
no plots are made
'''

%matplotlib inline 
%load_ext autoreload
%autoreload
import matplotlib
import matplotlib.pyplot as plt
from collections import OrderedDict
import itertools
import numpy as np
import numpy.ma as ma
import pandas as pd
import struct
import os
import xarray as xr
import glob
import datetime
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import seaborn as sns
np.seterr(divide='ignore', invalid='ignore')
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning) # not good to supress but they divide by nan are annoying
#warnings.simplefilter(action='ignore', category=UserWarning) # https://github.com/pydata/xarray/issues/2273
import json
from esio import EsioData as ed
from esio import ice_plot
from esio import import_data
import subprocess
import dask
from dask.distributed import Client
import timeit

# General plotting settings
sns.set_style('whitegrid')
sns.set_context("talk", font_scale=.8, rc={"lines.linewidth": 2.5})

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
# discovered that ukmetofficesipn march was actually some other month so 
# was not computing the metrics for march
# so tried to force it here
# once I get the data it should be possible to just run and get the metrics
# for that model only
# when done must adjust the end of cell a few down from here so that the other
# models are done again and so not update all 

In [10]:
#client = Client()
#client
dask.config.set(scheduler='threads')  # overwrite default with threaded scheduler

In [11]:
#def Update_PanArctic_Maps():
# Plotting Info
runType = 'forecast'
variables = ['sic']
metrics_all = {'sic':['anomaly','mean','SIP'], 'hi':['mean']}
#metrics_all = {'sic':['SIP']}

cvar = 'sic'

# Define Init Periods here, spaced by 7 days (aprox a week)
# Now
cd = datetime.datetime.now()
cd = datetime.datetime(cd.year, cd.month, cd.day) # Set hour min sec to 0. 

#cd = datetime.datetime(2019, 4, 1) # Set hour min sec to 0. 

print(cd)
# Hardcoded start date (makes incremental weeks always the same)
start_t = datetime.datetime(1950, 1, 1) # datetime.datetime(1950, 1, 1)
# Params for this plot
Ndays = 7 # time period to aggregate maps to (default is 7)

init_start_date = np.datetime64('2018-01-01') # first date we have computed metrics
                   # only updates files if missing or if forced with updateAll=True
#init_start_date = np.datetime64('2019-01-01') # speeds up substantially b
updateAll = False # if update all may wish to adjust init_start_date

# reduce this to spead things up 
init_start_date = np.datetime64('2019-10-01') # first date we have computed metrics
#updateAll = True

    
init_slice = np.arange(start_t, cd, datetime.timedelta(days=Ndays)).astype('datetime64[ns]')
# init_slice = init_slice[-Npers:] # Select only the last Npers of periods (weeks) since current date
init_slice = init_slice[init_slice>=init_start_date] # Select only the inits after init_start_date

print(init_slice[0],init_slice[-1])
print('')

# Forecast times
weeks = pd.to_timedelta(np.arange(0,52,1), unit='W')
#months = pd.to_timedelta(np.arange(2,12,1), unit='M')
#years = pd.to_timedelta(np.arange(1,2), unit='Y') - np.timedelta64(1, 'D') # need 364 not 365
#slices = weeks.union(months).union(years).round('1d')
fore_slice = xr.DataArray(weeks, dims=('fore_time'))
fore_slice.fore_time.values.astype('timedelta64[D]')
print(fore_slice)

2019-11-29 00:00:00
2019-06-02T00:00:00.000000000 2019-11-24T00:00:00.000000000

<xarray.DataArray (fore_time: 52)>
array([                0,   604800000000000,  1209600000000000,
        1814400000000000,  2419200000000000,  3024000000000000,
        3628800000000000,  4233600000000000,  4838400000000000,
        5443200000000000,  6048000000000000,  6652800000000000,
        7257600000000000,  7862400000000000,  8467200000000000,
        9072000000000000,  9676800000000000, 10281600000000000,
       10886400000000000, 11491200000000000, 12096000000000000,
       12700800000000000, 13305600000000000, 13910400000000000,
       14515200000000000, 15120000000000000, 15724800000000000,
       16329600000000000, 16934400000000000, 17539200000000000,
       18144000000000000, 18748800000000000, 19353600000000000,
       19958400000000000, 20563200000000000, 21168000000000000,
       21772800000000000, 22377600000000000, 22982400000000000,
       23587200000000000, 24192000000000000, 2479680

In [12]:
#############################################################
# Load in Observed and non-dynamic model Data
#############################################################

E = ed.EsioData.load()
mod_dir = E.model_dir

# Get median ice edge by DOY
median_ice_fill = xr.open_mfdataset(os.path.join(E.obs_dir, 'NSIDC_0051', 'agg_nc', 'ice_edge.nc')).sic
# Get mean sic by DOY
mean_1980_2010_sic = xr.open_dataset(os.path.join(E.obs_dir, 'NSIDC_0051', 'agg_nc', 'mean_1980_2010_sic.nc')).sic
# Get average sip by DOY
mean_1980_2010_SIP = xr.open_dataset(os.path.join(E.obs_dir, 'NSIDC_0051', 'agg_nc', 'hist_SIP_1980_2010.nc')).sip    

# Get recent observations
ds_81 = xr.open_mfdataset(E.obs['NSIDC_0081']['sipn_nc']+'_yearly/*.nc', concat_dim='time', autoclose=True, parallel=True)#,

# Define models to plot
models_2_plot = list(E.model.keys())
models_2_plot = [x for x in models_2_plot if x not in ['piomas','MME','MME_NEW','uclsipn','hcmr']] # remove some models
models_2_plot = [x for x in models_2_plot if E.icePredicted[x]] # Only predictive models
#models_2_plot = ['dampedAnomalyTrend']
#models_2_plot = ['ukmo']
#models_2_plot = ['ukmetofficesipn']
#models_2_plot = ['usnavysipn', 'usnavyncep', 'rasmesrl', 'noaasipn', 'noaasipn_ext', 'usnavygofs', 'modcansipns_3', 'modcansipns_4', 'szapirosipn', 'awispin', 'nicosipn']
#models_2_plot = ['fgoalssipn']
print(models_2_plot)
# climatology is done differently later in script

['dampedAnomalyTrend', 'gfdlsipn', 'yopp', 'ukmetofficesipn', 'ecmwfsipn', 'meteofrsipn', 'ecmwf', 'metreofr', 'ukmo', 'kma', 'ncep', 'usnavysipn', 'usnavyncep', 'rasmesrl', 'noaasipn', 'noaasipn_ext', 'usnavygofs', 'modcansipns_3', 'modcansipns_4', 'szapirosipn', 'awispin', 'nicosipn', 'fgoalssipn']


In [13]:
# def is_in_time_range(x):
    
#     if x.sel(init_time=slice(time_bds[0],time_bds[1])).init_time.size>0: # We have some time in the time range
#         return x
#     else:
#         return []
# time_bds = [init_slice[0],init_slice[-1]]

In [15]:
# will consider if we want to update all of these init times
#init_start_date = np.datetime64('2019-09-22') # first date we have computed metrics
#init_slice = init_slice[init_slice>=init_start_date] # Select only the inits after init_start_date

print(init_slice )

print(fore_slice)


['2019-09-22T00:00:00.000000000' '2019-09-29T00:00:00.000000000'
 '2019-10-06T00:00:00.000000000' '2019-10-13T00:00:00.000000000'
 '2019-10-20T00:00:00.000000000' '2019-10-27T00:00:00.000000000'
 '2019-11-03T00:00:00.000000000' '2019-11-10T00:00:00.000000000'
 '2019-11-17T00:00:00.000000000' '2019-11-24T00:00:00.000000000']
<xarray.DataArray (fore_time: 52)>
array([                0,   604800000000000,  1209600000000000,
        1814400000000000,  2419200000000000,  3024000000000000,
        3628800000000000,  4233600000000000,  4838400000000000,
        5443200000000000,  6048000000000000,  6652800000000000,
        7257600000000000,  7862400000000000,  8467200000000000,
        9072000000000000,  9676800000000000, 10281600000000000,
       10886400000000000, 11491200000000000, 12096000000000000,
       12700800000000000, 13305600000000000, 13910400000000000,
       14515200000000000, 15120000000000000, 15724800000000000,
       16329600000000000, 16934400000000000, 17539200000000000,

In [17]:
updateAll = True
updateAll = False

#models_2_plot=['fgoalssipn']
#models_2_plot=['meteofrsipn']
#updateAll = True


cvar = 'sic'

# Plot all Models
for cmod in models_2_plot:

    all_files = os.path.join(E.model[cmod][runType]['sipn_nc'], '*.nc') 
    print(all_files)

    # Check we have files 
    files = glob.glob(all_files)
    if not files:
        continue # Skip this model

    # only want to open files in the init_slice, do not want to use dask though 
    # for the idiotic cases 
        
    for it in init_slice: 
        it_start = it-np.timedelta64(Ndays,'D') + np.timedelta64(1,'D') # Start period for init period (it is end of period). Add 1 day because when
        # we select using slice(start,stop) it is inclusive of end points. So here we are defining the start of the init AND the start of the valid time.
        # So we need to add one day, so we don't double count.
        print('\n it_start  ',it_start,"to",it)

        # not actually using this but may want it later
        tmp = str(it_start.astype('datetime64[D]')).split('-')
        year0=tmp[0]
        month0=tmp[1]
        day0=tmp[2]
        tmp=str(it.astype('datetime64[D]')).split('-')
        year1=tmp[0]
        month1=tmp[1]
        day1=tmp[2]
        #print(year0,month0,day0, ' to ' ,year1,month1,day1)

        good_files = []
        for cfile in files:
        
            ds=xr.open_dataset(cfile,autoclose=True).init_time
            its = ds.values

            # Check if init_times are in range (works for list or single)
            dsinit =((its>=it_start) & (its<=it))
        
            if any(dsinit):
                good_files.append(cfile)
                
        if not good_files:
            print('No init_times found in range for model ',cmod)
            continue # Skip this init_time for this model

        #print('The files in range of interest are ',sorted(good_files))
        
        # Load in model for this init_time range, could include a few times not needed for 
        # models lumped by month
        drop_vars = [x for x in xr.open_dataset(sorted(good_files)[-1],autoclose=True).data_vars if x not in variables]
        ds_model_ALL = xr.open_mfdataset(sorted(good_files), 
                                     chunks={ 'fore_time': 1,'init_time': 1,'nj': 304, 'ni': 448},  
                                     concat_dim='init_time', autoclose=True, 
                                     parallel=True, drop_variables=drop_vars)
                                     # preprocess=lambda x : is_in_time_range(x)) # 'fore_time': 1, ,
        ds_model_ALL.rename({'nj':'x', 'ni':'y'}, inplace=True)

        # Sort by init_time
        ds_model_ALL = ds_model_ALL.sortby('init_time')

        # Get Valid time
        ds_model_ALL = import_data.get_valid_time(ds_model_ALL)
        print('possible init times for model ', cmod, 'are ',ds_model_ALL.init_time.values)

        # Select init period and fore_time of interest
        ds_model_ALL = ds_model_ALL.sel(init_time=slice(it_start, it))

        # Check we found any init_times in range
        if ds_model_ALL.init_time.size==0:
            print('init_time not found.')
            continue
            
        print('actual init times for model ', cmod, 'are ',ds_model_ALL.init_time.values)

        # Select var of interest (if available)
        if cvar in ds_model_ALL.variables:
            ds_model_ALL = ds_model_ALL[cvar]
        else:
            print('cvar not found.')
            continue

        print('**** JUST BEFORE ft loop, ds_model_ALL is ', ds_model_ALL)
        
        for ft in fore_slice.values: 

            cdoy_end = pd.to_datetime(it + ft).timetuple().tm_yday # Get current day of year end for valid time
            cdoy_start = pd.to_datetime(it_start + ft).timetuple().tm_yday  # Get current day of year end for valid time

            # Get datetime64 of valid time start and end
            valid_start = it_start + ft
            valid_end = it + ft
            
            # Check if we have any valid times in range of target dates
            ds_model = ds_model_ALL.where((ds_model_ALL.valid_time>=valid_start) & (ds_model_ALL.valid_time<=valid_end), drop=True) 
            print('valid_times are ', valid_start, valid_end)
            #print('ds_model ', ds_model)

            if ds_model.fore_time.size == 0:
                print("no fore_time found for target period.")
                continue

            ds_avg = None
            avg_done = False  # no avg yet
            
            # Loop through variable of interest + any metrics (i.e. SIP) based on that
            for metric in metrics_all[cvar]:

                # File paths and stuff
                out_metric_dir = os.path.join(E.model['MME_NEW'][runType]['sipn_nc'], cvar, metric)
                if not os.path.exists(out_metric_dir):
                    os.makedirs(out_metric_dir) 

                out_init_dir = os.path.join(out_metric_dir, pd.to_datetime(it).strftime('%Y-%m-%d'))
                if not os.path.exists(out_init_dir):
                    os.makedirs(out_init_dir)

                out_mod_dir = os.path.join(out_init_dir, cmod)
                if not os.path.exists(out_mod_dir):
                    os.makedirs(out_mod_dir)     

                out_nc_file = os.path.join(out_mod_dir, pd.to_datetime(it+ft).strftime('%Y-%m-%d')+'_'+cmod+'.nc')
#                    print((os.path.isfile(out_nc_file)), out_nc_file)

                # Only update if either we are updating All or it doesn't yet exist
                # or if one of the most recent init_times 
                if updateAll | (os.path.isfile(out_nc_file)==False) | np.any(it in init_slice[-2:]):
                    #print("    Updating...")
   
                    if not avg_done:
                        ds_avg = ds_model.mean(dim=['fore_time','init_time'])
                        #print('Got the average for computing metrics: ',metric)
                        avg_done = True

                    if metric=='mean': # Calc ensemble mean
                        ds_metric = ds_avg.mean(dim='ensemble')

                    elif metric=='SIP': # Calc probability
                        # Remove ensemble members having missing data
                        ok_ens = ((ds_avg.notnull().sum(dim='x').sum(dim='y'))>0) # select ensemble members with any data
                        ds_metric = ((ds_avg.where(ok_ens, drop=True)>=0.15) ).mean(dim='ensemble').where(ds_avg.isel(ensemble=0).notnull())

                    elif metric=='anomaly': # Calc anomaly in reference to mean observed 1980-2010
                        # Get climatological mean
                        da_obs_mean = mean_1980_2010_sic.isel(time=slice(cdoy_start,cdoy_end)).mean(dim='time')
                        # Calc anomaly
                        ds_metric = ds_avg.mean(dim='ensemble') - da_obs_mean
                        # Add back lat/long (get dropped because of round off differences)
                        ds_metric['lat'] = da_obs_mean.lat
                        ds_metric['lon'] = da_obs_mean.lon
                        
                    else:
                        raise ValueError('metric not implemented')

                    # drop ensemble if still present does not do it though
                    if 'ensemble' in ds_metric:
                        print('Getting rid of ensemble variable')
                        ds_metric = ds_metric.drop('ensemble')

                    ds_metric.coords['model'] = cmod
                    if 'xm' in ds_metric:
                        ds_metric = ds_metric.drop(['xm','ym']) #Dump coords we don't use

                    # Add Coords info
                    ds_metric.name = metric
                    ds_metric.coords['model'] = cmod
                    ds_metric.coords['init_start'] = it_start
                    ds_metric.coords['init_end'] = it
                    ds_metric.coords['valid_start'] = it_start+ft
                    ds_metric.coords['valid_end'] = it+ft
                    ds_metric.coords['fore_time'] = ft

                    # check if the data are missing 
                    if ds_metric.sum(dim=['x','y']).values == 0.:
                        print('Warning Metric is Undefined, no file written: ',out_nc_file)
                        if os.path.exists(out_nc_file):
                            os.remove(out_nc_file)
                        continue


                    #print('Done computing metric, our da is now: ', ds_metric)
                    #print('Saving to file: ',out_nc_file)
                    # Save to file
                    ds_metric.to_netcdf(out_nc_file)

                    #ds_metric.plot()
                    #plt.figure()


            #if avg_done:
            #    diehere

# Clean up 
ds_model = None
ds_avg = None
ds_metric = None

/home/disk/sipn/nicway/data/model/meteofrsipn/forecast/sipn_nc/*.nc

 it_start   2019-09-16T00:00:00.000000000 to 2019-09-22T00:00:00.000000000
No init_times found in range for model  meteofrsipn

 it_start   2019-09-23T00:00:00.000000000 to 2019-09-29T00:00:00.000000000
No init_times found in range for model  meteofrsipn

 it_start   2019-09-30T00:00:00.000000000 to 2019-10-06T00:00:00.000000000
possible init times for model  meteofrsipn are  ['2019-10-01T00:00:00.000000000']
actual init times for model  meteofrsipn are  ['2019-10-01T00:00:00.000000000']
**** JUST BEFORE ft loop, ds_model_ALL is  <xarray.DataArray 'sic' (init_time: 1, ensemble: 51, fore_time: 211, x: 304, y: 448)>
dask.array<shape=(1, 51, 211, 304, 448), dtype=float64, chunksize=(1, 51, 1, 304, 448)>
Coordinates:
    lon         (x, y) float64 168.3 168.4 168.5 168.7 ... -9.745 -9.872 -9.999
    lat         (x, y) float64 31.1 31.25 31.4 31.55 ... 34.92 34.77 34.62 34.47
  * ensemble    (ensemble) int32 0 1 2 3 4 5 6 

valid_times are  2019-12-16T00:00:00.000000000 2019-12-22T00:00:00.000000000
valid_times are  2019-12-23T00:00:00.000000000 2019-12-29T00:00:00.000000000
valid_times are  2019-12-30T00:00:00.000000000 2020-01-05T00:00:00.000000000
Warning Metric is Undefined, no file written:  /home/disk/sipn/nicway/data/model/MME_NEW/forecast/sipn_nc/sic/anomaly/2019-11-03/meteofrsipn/2020-01-05_meteofrsipn.nc
valid_times are  2020-01-06T00:00:00.000000000 2020-01-12T00:00:00.000000000
valid_times are  2020-01-13T00:00:00.000000000 2020-01-19T00:00:00.000000000
valid_times are  2020-01-20T00:00:00.000000000 2020-01-26T00:00:00.000000000
valid_times are  2020-01-27T00:00:00.000000000 2020-02-02T00:00:00.000000000
valid_times are  2020-02-03T00:00:00.000000000 2020-02-09T00:00:00.000000000
valid_times are  2020-02-10T00:00:00.000000000 2020-02-16T00:00:00.000000000
valid_times are  2020-02-17T00:00:00.000000000 2020-02-23T00:00:00.000000000
valid_times are  2020-02-24T00:00:00.000000000 2020-03-01T00:00

In [ ]:
# test
cfile='/home/disk/sipn/nicway/data/model/MME_NEW/forecast/sipn_nc/sic/anomaly/2019-08-25/fgoalssipn/2019-08-25_fgoalssipn.nc'
#cfile='/home/disk/sipn/nicway/data/model/MME_NEW/forecast/sipn_nc/sic/anomaly/2019-07-28/fgoalssipn/2019-12-01_fgoalssipn.nc'
cfile='/home/disk/sipn/nicway/data/model/MME_NEW/forecast/sipn_nc/sic/anomaly/2019-07-28/fgoalssipn/2019-12-08_fgoalssipn.nc'
cfile='/home/disk/sipn/nicway/data/model/MME_NEW/forecast/sipn_nc/sic/anomaly/2019-07-28/fgoalssipn/2019-10-06_fgoalssipn.nc'
ds=xr.open_mfdataset(cfile)
print(ds.anomaly.sum(dim=['x','y']).values)
#ds.anomaly.plot()

In [ ]:
skip = True   # this is the old way before tried to make it a bit faster without changing answers

if not skip:

    # Plot all Models
    for cmod in models_2_plot:
        print(cmod)

        # Load in Model
        # Find only files that have current year and month in filename (speeds up loading)
        all_files = os.path.join(E.model[cmod][runType]['sipn_nc'], '*.nc') 
    #    all_files = os.path.join(E.model[cmod][runType]['sipn_nc'], '*201906*.nc') 

        # Check we have files 
        files = glob.glob(all_files)
        if not files:
            continue # Skip this model

        # Get list of variablse we want to drop
        drop_vars = [x for x in xr.open_dataset(sorted(files)[-1],autoclose=True).data_vars if x not in variables]
        print('The files in range of interest are ',files)
        # Load in model   
        ds_model_ALL = xr.open_mfdataset(sorted(files), 
                                     chunks={ 'fore_time': 1,'init_time': 1,'nj': 304, 'ni': 448},  
                                     concat_dim='init_time', autoclose=True, 
                                     parallel=True, drop_variables=drop_vars)
                                     # preprocess=lambda x : is_in_time_range(x)) # 'fore_time': 1, ,
        ds_model_ALL.rename({'nj':'x', 'ni':'y'}, inplace=True)

        # Sort by init_time
        ds_model_ALL = ds_model_ALL.sortby('init_time')

        # Get Valid time
        ds_model_ALL = import_data.get_valid_time(ds_model_ALL)
        print('init times are ',ds_model_ALL.init_time.values)


        # For each variable
        for cvar in variables:

            # For each init time period
            for it in init_slice: 
                it_start = it-np.timedelta64(Ndays,'D') + np.timedelta64(1,'D') # Start period for init period (it is end of period). Add 1 day because when
                # we select using slice(start,stop) it is inclusive of end points. So here we are defining the start of the init AND the start of the valid time.
                # So we need to add one day, so we don't double count.
                print('it_start  ',it_start,"to",it)

                # For each forecast time we haven't plotted yet
                #ft_to_plot = ds_status.sel(init_time=it)
                #ft_to_plot = ft_to_plot.where(ft_to_plot.isnull(), drop=True).fore_time

                for ft in fore_slice.values: 

                    cdoy_end = pd.to_datetime(it + ft).timetuple().tm_yday # Get current day of year end for valid time
                    cdoy_start = pd.to_datetime(it_start + ft).timetuple().tm_yday  # Get current day of year end for valid time

                    # Get datetime64 of valid time start and end
                    valid_start = it_start + ft
                    valid_end = it + ft

                    # Loop through variable of interest + any metrics (i.e. SIP) based on that
                    for metric in metrics_all[cvar]:

                        # File paths and stuff
                        out_metric_dir = os.path.join(E.model['MME_NEW'][runType]['sipn_nc'], cvar, metric)
                        if not os.path.exists(out_metric_dir):
                            os.makedirs(out_metric_dir) 

                        out_init_dir = os.path.join(out_metric_dir, pd.to_datetime(it).strftime('%Y-%m-%d'))
                        if not os.path.exists(out_init_dir):
                            os.makedirs(out_init_dir)

                        out_mod_dir = os.path.join(out_init_dir, cmod)
                        if not os.path.exists(out_mod_dir):
                            os.makedirs(out_mod_dir)     

                        out_nc_file = os.path.join(out_mod_dir, pd.to_datetime(it+ft).strftime('%Y-%m-%d')+'_'+cmod+'.nc')
    #                    print((os.path.isfile(out_nc_file)), out_nc_file)

                        # Only update if either we are updating All or it doesn't yet exist
                        # OR, its one of the last 3 init times 
                        if updateAll | (os.path.isfile(out_nc_file)==False) | np.any(it in init_slice[-2:]):
                            #print("    Updating...")

                            # Select init period and fore_time of interest
                            ds_model = ds_model_ALL.sel(init_time=slice(it_start, it))

                            # Check we found any init_times in range
                            if ds_model.init_time.size==0:
                                #print('init_time not found.')
                                continue

                            # Select var of interest (if available)
                            if cvar in ds_model.variables:
                                ds_model = ds_model[cvar]
                            else:
                                #print('cvar not found.')
                                continue

                            # Check if we have any valid times in range of target dates
                            ds_model = ds_model.where((ds_model.valid_time>=valid_start) & (ds_model.valid_time<=valid_end), drop=True) 
                            if ds_model.fore_time.size == 0:
                                #print("no fore_time found for target period.")
                                continue
                            print('Averaging over fore_time and init_time for variable ds_model: ')
                            print(ds_model)
                            # Average over for_time and init_times
                            ds_model = ds_model.mean(dim=['fore_time','init_time'])

                            print('Got the average now computing metric: ',metric)


                            if metric=='mean': # Calc ensemble mean
                                ds_model = ds_model.mean(dim='ensemble')

                            elif metric=='SIP': # Calc probability
                                # Remove ensemble members having missing data
                                ok_ens = ((ds_model.notnull().sum(dim='x').sum(dim='y'))>0) # select ensemble members with any data
                                ds_model = ((ds_model.where(ok_ens, drop=True)>=0.15) ).mean(dim='ensemble').where(ds_model.isel(ensemble=0).notnull())

                            elif metric=='anomaly': # Calc anomaly in reference to mean observed 1980-2010
                                # Get climatological mean
                                da_obs_mean = mean_1980_2010_sic.isel(time=slice(cdoy_start,cdoy_end)).mean(dim='time')
                                # Calc anomaly
                                ds_model = ds_model.mean(dim='ensemble') - da_obs_mean
                                # Add back lat/long (get dropped because of round off differences)
                                ds_model['lat'] = da_obs_mean.lat
                                ds_model['lon'] = da_obs_mean.lon
                            else:
                                raise ValueError('metric not implemented')

                            print('Done computing metric, our da is now: ', ds_model)

                            # drop ensemble if still present
                            if 'ensemble' in ds_model:
                                ds_model = ds_model.drop('ensemble')

                            ds_model.coords['model'] = cmod
                            if 'xm' in ds_model:
                                ds_model = ds_model.drop(['xm','ym']) #Dump coords we don't use

                            # Add Coords info
                            ds_model.name = metric
                            ds_model.coords['model'] = cmod
                            ds_model.coords['init_start'] = it_start
                            ds_model.coords['init_end'] = it
                            ds_model.coords['valid_start'] = it_start+ft
                            ds_model.coords['valid_end'] = it+ft
                            ds_model.coords['fore_time'] = ft

                            # Save to file
                            ds_model.to_netcdf(out_nc_file)

                            # Clean up for current model
                            ds_model = None


In [ ]:
###########################################################
#          climatology  trend                             #
###########################################################

cmod = 'climatology'

all_files = os.path.join(mod_dir,cmod,runType,'sipn_nc', '*.nc')
files = glob.glob(all_files)

obs_clim_model = xr.open_mfdataset(sorted(files), 
        chunks={'time': 30, 'x': 304, 'y': 448},  
         concat_dim='time', autoclose=True, parallel=True)

In [ ]:
# For each variable
for cvar in variables:

    # For each init time period
    for it in init_slice: 
        it_start = it-np.timedelta64(Ndays,'D') + np.timedelta64(1,'D') # Start period for init period (it is end of period). Add 1 day because when
        # we select using slice(start,stop) it is inclusive of end points. So here we are defining the start of the init AND the start of the valid time.
        # So we need to add one day, so we don't double count.
        print(it_start,"to",it)

        for ft in fore_slice.values: 

            cdoy_end = pd.to_datetime(it + ft).timetuple().tm_yday # Get current day of year end for valid time
            cdoy_start = pd.to_datetime(it_start + ft).timetuple().tm_yday  # Get current day of year end for valid time

            # Get datetime64 of valid time start and end
            valid_start = it_start + ft
            valid_end = it + ft
#            print('valid_end ', valid_end)

            # Loop through variable of interest + any metrics (i.e. SIP) based on that
            for metric in metrics_all[cvar]:

                # File paths and stuff
                out_metric_dir = os.path.join(E.model['MME_NEW'][runType]['sipn_nc'], cvar, metric)
                if not os.path.exists(out_metric_dir):
                    os.makedirs(out_metric_dir) 

                out_init_dir = os.path.join(out_metric_dir, pd.to_datetime(it).strftime('%Y-%m-%d'))
                if not os.path.exists(out_init_dir):
                    os.makedirs(out_init_dir)

                out_mod_dir = os.path.join(out_init_dir, cmod)
                if not os.path.exists(out_mod_dir):
                    os.makedirs(out_mod_dir)     

                out_nc_file = os.path.join(out_mod_dir, pd.to_datetime(it+ft).strftime('%Y-%m-%d')+'_'+cmod+'.nc')

                # Only update if either we are updating All or it doesn't yet exist
                # OR, its one of the last 3 init times 
                if updateAll | (os.path.isfile(out_nc_file)==False) | np.any(it in init_slice[-2:]):
                    #print("    Updating...")

                    # Check if we have any valid times in range of target dates
                    ds_model = obs_clim_model[cvar].where((obs_clim_model.time>=valid_start) & (obs_clim_model.time<=valid_end), drop=True) 
                    if 'time' in ds_model.lat.dims:
                        ds_model.coords['lat'] = ds_model.lat.isel(time=0).drop('time') # Drop time from lat/lon dims (not sure why?)

                    # If we have any time
                    if ds_model.time.size > 0:

                        # Average over time
                        ds_model = ds_model.mean(dim='time')

                        if metric=='mean': # Calc ensemble mean
                            ds_model = ds_model
                        elif metric=='SIP': # Calc probability
                            # Issue of some ensemble members having missing data
                            ocnmask = ds_model.notnull()
                            ds_model = (ds_model>=0.15).where(ocnmask)
                        elif metric=='anomaly': # Calc anomaly in reference to mean observed 1980-2010
                            # Get climatological mean
                            da_obs_mean = mean_1980_2010_sic.isel(time=slice(cdoy_start,cdoy_end)).mean(dim='time')
                            # Get anomaly
                            ds_model = ds_model - da_obs_mean
                            # Add back lat/long (get dropped because of round off differences)
                            ds_model['lat'] = da_obs_mean.lat
                            ds_model['lon'] = da_obs_mean.lon
                        else:
                            raise ValueError('metric not implemented')   

                        # Drop un-needed coords to match model format
                        if 'doy' in ds_model.coords:
                            ds_model = ds_model.drop(['doy'])
                        if 'xm' in ds_model.coords:
                            ds_model = ds_model.drop(['xm'])
                        if 'ym' in ds_model.coords:
                            ds_model = ds_model.drop(['ym'])
                    
                        # Add Coords info
                        ds_model.name = metric
                        ds_model.coords['model'] = cmod
                        ds_model.coords['init_start'] = it_start
                        ds_model.coords['init_end'] = it
                        ds_model.coords['valid_start'] = it_start+ft
                        ds_model.coords['valid_end'] = it+ft
                        ds_model.coords['fore_time'] = ft
                        
                        # Save to file
                        ds_model.to_netcdf(out_nc_file)

                        # Clean up for current model
                        ds_model = None
                        
obs_clim_model = None


In [ ]:
############################################################################
#                               OBSERVATIONS                               #
############################################################################

cmod = 'Observed'

updateAll = True # We ALWAYS want to update all observations for recent past

# make init_start_date about 1 mon ago

init_start_date = np.datetime64('today') - np.timedelta64(32,'D')

print('init_start_date ',init_start_date)
    
init_slice = np.arange(start_t, cd, datetime.timedelta(days=Ndays)).astype('datetime64[ns]')
init_slice = init_slice[init_slice>=init_start_date] # Select only the inits after init_start_date

print(init_slice[0],init_slice[-1])


# For each variable
for cvar in variables:

    # For each init time period
    for it in init_slice: 
        it_start = it-np.timedelta64(Ndays,'D') + np.timedelta64(1,'D') # Start period for init period (it is end of period). Add 1 day because when
        # we select using slice(start,stop) it is inclusive of end points. So here we are defining the start of the init AND the start of the valid time.
        # So we need to add one day, so we don't double count.
        print(it_start,"to",it)

        for ft in fore_slice.values: 

            cdoy_end = pd.to_datetime(it + ft).timetuple().tm_yday # Get current day of year end for valid time
            cdoy_start = pd.to_datetime(it_start + ft).timetuple().tm_yday  # Get current day of year end for valid time

            # Get datetime64 of valid time start and end
            valid_start = it_start + ft
            valid_end = it + ft

            # Loop through variable of interest + any metrics (i.e. SIP) based on that
            for metric in metrics_all[cvar]:

                # File paths and stuff
                out_metric_dir = os.path.join(E.model['MME_NEW'][runType]['sipn_nc'], cvar, metric)
                if not os.path.exists(out_metric_dir):
                    os.makedirs(out_metric_dir) 

                out_init_dir = os.path.join(out_metric_dir, pd.to_datetime(it).strftime('%Y-%m-%d'))
                if not os.path.exists(out_init_dir):
                    os.makedirs(out_init_dir)

                out_mod_dir = os.path.join(out_init_dir, cmod)
                if not os.path.exists(out_mod_dir):
                    os.makedirs(out_mod_dir)     

                out_nc_file = os.path.join(out_mod_dir, pd.to_datetime(it+ft).strftime('%Y-%m-%d')+'_'+cmod+'.nc')

                # Only update if either we are updating All or it doesn't yet exist
                # OR, its one of the last 3 init times 
                if updateAll | (os.path.isfile(out_nc_file)==False) | np.any(it in init_slice[-2:]):
                    #print("    Updating...")

                    # Check if we have any valid times in range of target dates
                    ds_model = da_obs_c = ds_81[cvar].sel(time=slice(valid_start, valid_end))
                    
                    if 'time' in ds_model.lat.dims:
                        ds_model.coords['lat'] = ds_model.lat.isel(time=0).drop('time') # Drop time from lat/lon dims (not sure why?)

                    # Check we have all observations for this week (7)
                    if ds_model.time.size == 7:

                        if metric=='mean':
                            ds_model = ds_model.mean(dim='time') #ds_81.sic.sel(time=(it + ft))
                        elif metric=='SIP':
                            ds_model = (ds_model >= 0.15).mean(dim='time').astype('int').where(ds_model.isel(time=0).notnull())
                        elif metric=='anomaly':
                            da_obs_VT = ds_model.mean(dim='time')
                            da_obs_mean = mean_1980_2010_sic.isel(time=slice(cdoy_start,cdoy_end)).mean(dim='time')
                            ds_model = da_obs_VT - da_obs_mean
                        else:
                            raise ValueError('Not implemented')

                        # Drop coords we don't need
                        ds_model = ds_model.drop(['hole_mask','xm','ym'])
                        if 'time' in ds_model:
                            ds_model = ds_model.drop('time')

                        # Add Coords info
                        ds_model.name = metric
                        ds_model.coords['model'] = cmod
                        ds_model.coords['init_start'] = it_start
                        ds_model.coords['init_end'] = it
                        ds_model.coords['valid_start'] = it_start+ft
                        ds_model.coords['valid_end'] = it+ft
                        ds_model.coords['fore_time'] = ft

                        # Write to disk
                        ds_model.to_netcdf(out_nc_file)

                        # Clean up for current model
                        ds_model = None

In [ ]:
print("Finished Calculating Weekly means")

In [ ]:
# from dask.distributed import Client
# client = Client(n_workers=8)
# client

In [ ]:
# cvar = 'sic' # hard coded for now
# # Load in all data and write to Zarr
# # Load in all metrics for given variable
# print("Loading in weekly metrics...")
# ds_m = import_data.load_MME_by_init_end(E=E, 
#                                         runType=runType, 
#                                         variable=cvar, 
#                                         metrics=metrics_all[cvar])

# # Drop models that we don't evaluate (i.e. monthly means)
# models_keep = [x for x in ds_m.model.values if x not in ['noaasipn','modcansipns_3','modcansipns_4']]
# ds_m = ds_m.sel(model=models_keep)
# # Get list of dynamical models that are not observations
# dynamical_Models = [x for x in ds_m.model.values if x not in ['Observed','climatology','dampedAnomaly','dampedAnomalyTrend']]
# # # Get list of all models
# # all_Models = [x for x in ds_m.model.values if x not in ['Observed']]
# # Add MME
# MME_avg = ds_m.sel(model=dynamical_Models).mean(dim='model') # only take mean over dynamical models
# MME_avg.coords['model'] = 'MME'
# ds_ALL = xr.concat([ds_m, MME_avg], dim='model')

In [ ]:

# ###### ADD METADATA #################

# ## Add coordinate system info
# ds_ALL.coords['crs'] = xr.DataArray('crs')
# ds_ALL['crs'].attrs = {
#     'comment': '(https://nsidc.org/data/polar-stereo/ps_grids.html or https://nsidc.org/data/oib/epsg_3413.html) This is a container variable that describes the grid_mapping used by the data in this file. This variable does not contain any data; only information about the geographic coordinate system',
#     'grid_mapping_name': 'polar_stereographic',
#     'straight_vertical_longitude_from_pole':'-45',
#     'latitude_of_projection_origin': '90.0',
#     'standard_parallel':'70',
#     'false_easting':'0',
#     'false_northing':'0'
#     }

# # Add time coords
# ds_ALL.coords['init_start'] = ds_ALL.init_end - np.timedelta64(Ndays,'D') + np.timedelta64(1,'D')
# ds_ALL['init_start'].attrs = {
#     'comment':        'Start date for weekly average period',
#     'long_name':      'Start date for weekly average period',
#     'standard_name':  "start_init_date"}

# ds_ALL['init_end'].attrs = {
#     'comment':        'End date for weekly average period',
#     'long_name':      'End date for weekly average period',
#     'standard_name':  "end_init_date"}

# ds_ALL['fore_time'].attrs = {
#     'comment':        'Forecast lead time',
#     'long_name':      'Forecast lead time',
#     'standard_name':  "forecast_lead_time"}

# # Add Valid time (start and end period)
# ds_ALL = import_data.get_valid_time(ds_ALL, init_dim='init_end', fore_dim='fore_time')
# ds_ALL.rename({'valid_time':'valid_end'}, inplace=True);
# ds_ALL.coords['valid_start'] = ds_ALL.valid_end - np.timedelta64(Ndays,'D') + np.timedelta64(1,'D')

# # Add attributes
# ds_ALL['valid_end'].attrs = {
#     'comment':        'End Valid date for weekly average period',
#     'long_name':      'End Valid date for weekly average period',
#     'standard_name':  "end_valid_date"}

# ds_ALL['valid_start'].attrs = {
#     'comment':        'Start Valid date for weekly average period',
#     'long_name':      'Start Valid date for weekly average period',
#     'standard_name':  "start_valid_date"}

# # Add Variable attributes
# ds_ALL['SIP'].attrs = {
#     'comment':        'Sea ice probability, calculated by averaging across ensemble members predictions of sea ice concentration >= 0.15',
#     'grid_mapping':   'crs',
#     'long_name':      'Sea ice probability',
#     'standard_name':  "sea_ice_probability",
#     'units':          'fraction'}

# ds_ALL['anomaly'].attrs = {
#     'comment':        'Anomaly of the forecasted sea ice concentration mean (ensemble average) compared to the 1980 to 2010 Observed Climatology',
#     'grid_mapping':   'crs',
#     'long_name':      'Anomaly',
#     'standard_name':  "anomaly",
#     'units':          'fraction'}

# ds_ALL['mean'].attrs = {
#     'comment':        'Mean of the forecasted sea ice concentration (ensemble average)',
#     'grid_mapping':   'crs',
#     'long_name':      'Sea ice concentration',
#     'standard_name':  "sea_ice_concentration",
#     'units':          'fraction'}

# # Dataset Attributes
# ds_ALL.attrs = {
# 'comment':                         'Weekly mean sea ice concentration forecasted by multiple models as well as observed by remotly sensed passive microwave sensors.',
# 'contact':                         'nicway@uw.edu',
# 'creator_email':                   'nicway@uw.edu',
# 'creator_name':                    'Nicholas Wayand, University of Washington',
# 'creator_url':                     'https://atmos.uw.edu/sipn/',
# 'date_created':                    '2018-12-03T00:00:00',
# 'date_modified':                   datetime.datetime.now().strftime('%Y-%m-%dT%H:%M:%S'),
# 'geospatial_lat_max':              str(float(ds_ALL.lat.max().values)),
# 'geospatial_lat_min':              str(float(ds_ALL.lat.min().values)),
# 'geospatial_lat_resolution':       '~25km',
# 'geospatial_lat_units':            'degrees_north',
# 'geospatial_lon_max':              str(float(ds_ALL.lon.max().values)),
# 'geospatial_lon_min':              str(float(ds_ALL.lon.min().values)),
# 'geospatial_lon_resolution':       '~25km',
# 'geospatial_lon_units':            'degrees_east',
# 'history':                         datetime.datetime.now().strftime('%Y-%m-%dT%H:%M:%S')+': updated by Nicholas Wayand',
# 'institution':                     'UW, SIPN, ARCUS',
# 'keywords':                        'Arctic > Sea ice concentration > Prediction',
# 'product_version':                 '1.0',
# 'project':                         'Sea Ice Prediction Network Phase II',
# 'references':                      'Wayand, N.E., Bitz, C.M., and E. Blanchard-Wrigglesworth, (in review). A year-round sub-seasonal to seasonal sea ice prediction portal. Submited to Geophysical Research letters.',
# 'source':                          'Numerical model predictions and Passive microwave measurments.',
# 'summary':                         'Dataset is updated daily with weekly sea ice forecasts',
# 'time_coverage_end':               pd.to_datetime(ds_ALL.valid_end.max().values).strftime('%Y-%m-%dT%H:%M:%S'),
# 'time_coverage_start':             pd.to_datetime(ds_ALL.init_start.min().values).strftime('%Y-%m-%dT%H:%M:%S'),
# 'title':                           'SIPN2 Sea ice Concentration Forecasts and Observations.'
# }

# ####################################
# print(ds_ALL)


# # Rechunk from ~1MB to 100MB chunks
# # Chunk along fore_time and init_end
# ds_ALL = ds_ALL.chunk({'fore_time': 10, 'init_end': 10, 'model': 1, 'x': 304, 'y': 448})

# # Save to Zarr
# print("Saving to Zarr...")
# ds_ALL.to_zarr(os.path.join(E.data_dir,'model/zarr', cvar+'.zarr'), mode='w')
# print("Finished updating Weekly SIC metrics and saved to Zar")
# ds_ALL=None # Flush memory
# MME_avg=None
# ds_m=None